In [1]:
import numpy as np
import pandas as pd
import math
import random
import subprocess
from multiprocessing.pool import ThreadPool
import multiprocessing as mp
import pickle

In [2]:
S, S_inv, p = [[3, 8, 11, 5, 6, 4, 14, 10, 2, 12, 1, 7, 9, 15, 13, 0], [15, 10, 8, 0, 5, 3, 4, 11, 1, 12, 7, 2, 9, 14, 6, 13], [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15 ]]

In [3]:
r, n, n2 = [6, 4, 16]

In [9]:
roundKeys = [random.randint(0, 2**16) for _ in range(r + 1)]
print(roundKeys)

[4900, 10493, 19513, 48196, 20715, 20640, 34695]


In [4]:
def p_shift(x): 
    y = sum(1 << p[j] for j in range(n2) if ((x >> j) & 1) == 1)
    return y


def p_shift_rev(x):  
    y = sum([1 << j for j in range(n2) if ((x >> p[j]) & 1) == 1])
    return y


def S_f(x):  
    y = sum([S[(x & (15 << (4*i))) >> (4*i)] << (4*i) for i in range(4)])
    x = y
    y = 0
    return x


def S_inv_f(x):  
    y = sum([S_inv[(x & (15 << (4*i))) >> (4*i)] << (4*i) for i in range(4)])
    x = y
    y = 0
    return x

In [8]:
def Enc(x, print_info=False):  
    i = 0
    while i < r:
        x[0] ^= roundKeys[i]
        x[0] = sum([S[(x[0] & (15 << (4*j))) >> (4*j)] << (4*j) for j in range(4)])
        x[1] = 0

        j = 0
        while j < n2:
            if ((x[0] >> j) & 1) == 1:
                x[1] |= 1 << p[j]
            j += 1

        x[0] = x[1]
        x[1] = 0

        if print_info:
            print(f"------- Round {i+1} -------")
            print(f"key = {roundKeys[i]}")
            print(f"y = {x[0]}")

        i += 1

    x[0] ^= roundKeys[r]
    x[1] = 0

    if print_info:
        print(f"------- Round {r+1} -------")
        print(f"key = {roundKeys[r]}")
        print(f"y = {x[0]}")

    return x

In [11]:
def Dec(x, print_info=False): 
    x[0] ^= roundKeys[r]
    if print_info:
        print(f"------- Round {r} -------")
        print(f"key = {roundKeys[r]}")
        print(f"y = {x[0]}")

    i = 0
    while i < r:
        x[1] = 0
        j = 0
        while j < n2:
            if ((x[0] >> p[j]) & 1) == 1:
                x[1] |= 1 << j
            j += 1
        x[0] = x[1]
        x[1] = 0
        x[1] = sum([S_inv[(x[0] & (15 << (4*j))) >> (4*j)] << (4*j) for j in range(4)])
        x[0] = x[1]

        x[0] ^= roundKeys[r-i-1]
        if print_info:
            print(f"------- Round {r-i-1} -------")
            print(f"key = {roundKeys[r-i-1]}")
            print(f"y = {x[0]}")

        i += 1

    x[1] = 0
    return x

In [12]:
def mul(x, alpha): 
    res = 0
    i = 0
    while i < n2:
        temp1 = ((x >> i) & 1)
        temp2 = ((alpha >> i) & 1)
        res ^= temp1 * temp2
        i += 1
    return res

In [24]:
%%time
j = 0
while j<10000:
    a = mul(2285, 41765)
    j+=1

CPU times: total: 15.6 ms
Wall time: 15.6 ms


In [26]:
def tableLP(): 
    LP = [[0 for i in range(n2)] for i in range(n2)]
    LP[0][0] = 1
    alpha = 1
    while alpha < n2:
        beta = 1
        while beta < n2:
            x = 0
            while x < n2:
                sx = S_f(x)
                a_x = mul(x, alpha)
                b_sx = mul(sx, beta)
                LP[alpha][beta] += (-1) ** (a_x ^ b_sx)
                x += 1
            
            LP[alpha][beta] = (LP[alpha][beta]/n2)**2
            beta += 1
        alpha += 1
    return LP

In [28]:
LP=tableLP()

pd.DataFrame(LP)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0,0.0625,0.0625,0.2500,0.0625,0.2500,0.0000,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.0000,0.0625
2,0,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.2500,0.0625
3,0,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.2500
4,0,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.0625,0.2500,0.0625,0.2500,0.0000,0.0625,0.0000,0.0625
5,0,0.0000,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000
6,0,0.2500,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0000,0.0000
7,0,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625
8,0,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000
9,0,0.0000,0.2500,0.2500,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625


In [29]:
LP = np.array(LP).transpose()

In [30]:
def LP_a(alpha): 
    a = [int(i, 16) for i in hex(alpha)[2:].zfill(n)]
    beta = [[j for j in range(n2) if LP[j][a[i]] != 0] for i in range(4)]
    LP_alpha = [[beta[3][block4] + (beta[2][block3] << 4) + (beta[1][block2] << 8) + (beta[0][block1] << 12),
                 LP[beta[0][block1]][a[0]] * LP[beta[1][block2]][a[1]] * LP[beta[2][block3]][a[2]] * LP[beta[3][block4]][a[3]]]
                for block1 in range(len(beta[0]))
                for block2 in range(len(beta[1]))
                for block3 in range(len(beta[2]))
                for block4 in range(len(beta[3]))]
    return LP_alpha

In [31]:
%%time
LP_a(5255)

CPU times: total: 15.6 ms
Wall time: 21.8 ms


[[4369, 1.52587890625e-05],
 [4370, 1.52587890625e-05],
 [4372, 6.103515625e-05],
 [4373, 1.52587890625e-05],
 [4374, 1.52587890625e-05],
 [4376, 1.52587890625e-05],
 [4379, 1.52587890625e-05],
 [4380, 1.52587890625e-05],
 [4382, 6.103515625e-05],
 [4383, 1.52587890625e-05],
 [4385, 1.52587890625e-05],
 [4386, 1.52587890625e-05],
 [4388, 6.103515625e-05],
 [4389, 1.52587890625e-05],
 [4390, 1.52587890625e-05],
 [4392, 1.52587890625e-05],
 [4395, 1.52587890625e-05],
 [4396, 1.52587890625e-05],
 [4398, 6.103515625e-05],
 [4399, 1.52587890625e-05],
 [4433, 1.52587890625e-05],
 [4434, 1.52587890625e-05],
 [4436, 6.103515625e-05],
 [4437, 1.52587890625e-05],
 [4438, 1.52587890625e-05],
 [4440, 1.52587890625e-05],
 [4443, 1.52587890625e-05],
 [4444, 1.52587890625e-05],
 [4446, 6.103515625e-05],
 [4447, 1.52587890625e-05],
 [4449, 1.52587890625e-05],
 [4450, 1.52587890625e-05],
 [4452, 6.103515625e-05],
 [4453, 1.52587890625e-05],
 [4454, 1.52587890625e-05],
 [4456, 1.52587890625e-05],
 [4459

In [32]:
pr = 0.0001

In [33]:

def LPSearch(alpha): 
    G_alpha = [dict() for _ in range(r)]
    G_alpha[0][alpha] = 1
    lps = {}
    for t in range(r)[1:]:
        g_a = {}
        for beta in G_alpha[t-1]:
            lp = lps.get(beta, LP_a(beta))
            lps[beta] = lp
            p_i = G_alpha[t-1][beta]
            for pair_lp in lp:
                g_a[pair_lp[0]] = g_a.get(pair_lp[0], 0) + pair_lp[1] * p_i
        G_alpha[t].update({p_shift(beta): g_a[beta] for beta in g_a if g_a[beta] > pr})
    return G_alpha[-1]

In [35]:
%%time   

LPs = {i << (4 * j): LPSearch(i << (4 * j)) for i in range(1, n2) for j in range(n) if len(LPSearch(i << (4 * j))) != 0}

CPU times: total: 17min 27s
Wall time: 1h 13min 20s


In [36]:
lp_list = [[k, b, LPs[k][b]] for k in LPs.keys() for b in LPs[k].keys()]

In [37]:
lp_list.sort(key=lambda x: x[2])

In [39]:
with open("lp_list", "wb") as fp: 
    pickle.dump(lp_list, fp)

In [14]:
with open("lp_list", "rb") as fp:   
    lp_list = pickle.load(fp)

In [15]:
lp_list = lp_list[::-1]  
print(len(lp_list))

lp_list = lp_list[:300]

In [17]:
import pprint

pprint.pprint(lp_list)

[[6, 68, 0.00041090650483965874],
 [7, 68, 0.0003769392005779082],
 [6, 16448, 0.0003510796232149005],
 [24576, 17, 0.0003435893449932337],
 [1536, 34, 0.0003415507962927222],
 [6, 136, 0.0003397067484911531],
 [28672, 17, 0.0003296284767202451],
 [12, 16448, 0.0003283520054537803],
 [6, 1028, 0.00032416137219115626],
 [14, 68, 0.00032401486532762647],
 [24576, 4112, 0.0003193524607922882],
 [1792, 34, 0.000318411552143516],
 [5, 68, 0.0003141236702504102],
 [1536, 8224, 0.00030386121943593025],
 [49152, 4112, 0.0002985541505040601],
 [7, 136, 0.0002981241559609771],
 [6, 32896, 0.00029245042242109776],
 [3072, 8224, 0.00028532149735838175],
 [5, 1092, 0.00028385830228216946],
 [6, 1024, 0.00028361856675473973],
 [20480, 17, 0.00028191641649755184],
 [8, 68, 0.0002808927674777806],
 [12, 68, 0.00027887264150194824],
 [12, 32896, 0.0002746209211181849],
 [6, 17476, 0.00027403095555200707],
 [6, 2056, 0.00027305822004564106],
 [14, 136, 0.0002719356562010944],
 [1280, 34, 0.0002707877029

In [19]:
num_texts = 4100
print(num_texts)

4100

In [20]:
M = []
i = 0

while i < num_texts:
    M.append(random.randint(1, 2**16 - 1))
    i += 1

C = []
i = 0

while i < num_texts:
    C.append(Enc([M[i], 0])[0])
    i += 1

key_candidate = []
i = 1

while i < 2**16 - 1:
    key_candidate.append(i)
    i += 1

In [23]:
def one_round(m, k): 
    x = m ^ k
    y = sum(S[(x >> (i * 4)) & 0xF] << (i * 4) for i in range(4))
    x = y
    y = sum(1 << p[j] for j in range(n2) if (x >> j) & 1)
    x = y
    return x

In [25]:
one_round_encrypted_m = dict()

In [26]:
%%time
one_round_encrypted_m = {k: [one_round(M[m], k) for m in range(num_texts)] for k in key_candidate}

CPU times: total: 6min 21s
Wall time: 6min 38s


In [29]:
with open("one_round_encrypted_m", "wb") as fp:   
    pickle.dump(one_round_encrypted_m, fp)

In [30]:
%%time
bY = {}
i = 0

while i < len(lp_list):
    b = lp_list[i]
    bY[b[1]] = [mul(b[1], c) for c in range(2**16)]
    i += 1

CPU times: total: 21.6 s
Wall time: 6min 42s


In [31]:
%%time
aX = {}
i = 0

while i < len(lp_list):
    a = lp_list[i]
    aX[a[0]] = [mul(a[0], x) for x in range(2**16)]
    i += 1

CPU times: total: 19.5 s
Wall time: 20.2 s


In [32]:
data = {"M": M, "C": C, "aX": aX, "bY": bY}

for filename, variable in data.items():
    with open(filename, "wb") as fp:
        pickle.dump(variable, fp)

In [33]:
M, C, aX, bY, one_round_encrypted_m = [pickle.load(open(filename, "rb")) for filename in ["M", "C", "aX", "bY", "one_round_encrypted_m"]]


In [34]:
num_texts = len(M)

In [35]:
import os
os.cpu_count()

8

In [36]:
key_candidate = list(range(1, 2**16-1))

counter = np.zeros(2**16, dtype=int)

In [38]:
def u_k(x1, c, a, b):
    uk = sum(1 if (aX[a][x1[i]] ^ bY[b][c[i]]) == 0 else -1 for i in range(num_texts))
    return abs(uk)

In [40]:
def m2_process(lp):
    if vocal:
        print(lp)
    
    u = [u_k(one_round_encrypted_m[key], C, lp[0], lp[1]) for key in key_candidate]
    
    max_u = max(u)
    if vocal:
        print("u =", max_u)
        
    for key in key_candidate:
        if u[key] == max_u:
            counter[key]+=1
    
    return True

In [41]:
lp_first_half = lp_list[:50]

In [42]:
with open("input_lab_2.bin", 'wb') as f:
    for m in M:
        f.write(m.to_bytes(2, 'little'))

subprocess.Popen("Heys.exe e 7 input_lab_2.bin output_lab_2.bin", 
                 stdin = subprocess.PIPE, shell = True).communicate(timeout = 5)           


(None, None)

In [43]:
C_hayes = []

with open("output_lab_2.bin", 'rb') as f:
    enc = f.read()
    C_i = int.from_bytes(enc, byteorder='little')
    
    i = 0
    while i < len(enc) // 2:
        temp1 = (C_i >> (i * 16))
        temp2 = (2 ** 16 - 1)
        C_hayes.append(temp1 & temp2)
        i += 1
        
C = C_hayes

In [45]:
%%time
vocal = False
with ThreadPool() as pool:
    pool.map(m2_process, lp_list[:50])
    pool.close()
    pool.join()

CPU times: total: 14min 22s
Wall time: 1h 28min 20s


In [46]:
with open("counter", "wb") as fp:   
    pickle.dump(counter, fp)

In [47]:
key_candidate = []
i = 1

while i < 2**16 - 1:
    if counter[i] > 0:
        key_candidate.append(i)
    i += 1

print(len(key_candidate))

19663

In [48]:
%%time
vocal = False
with ThreadPool() as pool:
    pool.map(m2_process, lp_list[50:])
    pool.close()
    pool.join()

CPU times: total: 20min 38s
Wall time: 9h 23min 27s


In [50]:
(-counter).argsort()[:10]

array([56071, 56263, 56311, 56183, 56167, 56295, 56103, 56066, 56199,
       21234], dtype=int64)

In [52]:
print(counter[56295])
for i in (-counter).argsort()[:5]:
    print(counter[i])

14
15
14
14
14
14
